In [1]:
import torch

In [13]:
class ModelTrain():
    def __init__(self, config):
        self.config = config

    def train_mogrifier_lstm(self, model, optimizer_model, loss_function, real_train, real_test, step_per_epoch, helper):
    
        loss_evol = []
        real_dataset = torch.empty((0, self.config.future_step, self.config.input_size), dtype=torch.float32).to(self.config.device)
        gen_dataset = torch.empty((0, self.config.future_step, self.config.input_size), dtype=torch.float32).to(self.config.device)
        epoch = 400
        
        for i in range(epoch):
            count = 0
            epoch_loss = 0.0

            for step in range(step_per_epoch):
                
                real_data = helper.gen_batch_forecasting(self.config.batch_size, count, real_train)
                real_label = helper.gen_batch_forecasting(self.config.batch_size, count, real_test)

                model.zero_grad()
                generated_samples = model(real_data)

                L2 = torch.sum((real_label - generated_samples)**2)
                g_loss = loss_function(generated_samples, real_label) + L2
                g_loss.backward()
                epoch_loss += g_loss.item()

                optimizer_model.step()
                
                count += 1
                if i == epoch - 1:
                    real_dataset = torch.cat([real_dataset, real_label], dim=0)
                    gen_dataset = torch.cat([gen_dataset, generated_samples], dim=0)
            
            epoch_loss /= step_per_epoch # average loss per batch
            loss_evol.append(epoch_loss)
            
            print(f"epoch: {i}, Train loss: {epoch_loss:.7f}")

        return real_dataset, gen_dataset, loss_evol
    
    def train_bi_lstm(self, model, optimizer_model, loss_function, real_train, real_test, step_per_epoch, helper):
    
        loss_evol = []
        real_dataset = torch.empty((0, self.config.future_step, self.config.input_size), dtype=torch.float32).to(self.config.device)
        gen_dataset = torch.empty((0, self.config.future_step, self.config.input_size), dtype=torch.float32).to(self.config.device)
        epoch = 300
        
        for i in range(epoch):
            count = 0
            epoch_loss = 0.0

            for step in range(step_per_epoch):
                # Data for training the discriminator
                real_data = helper.gen_batch_forecasting(self.config.batch_size, count, real_train)
                real_label = helper.gen_batch_forecasting(self.config.batch_size, count, real_test)

                model.zero_grad()
                generated_samples = model(real_data)

                L2 = torch.sum((real_label - generated_samples)**2)
                g_loss = loss_function(generated_samples, real_label) + L2
                g_loss.backward()
                epoch_loss += g_loss.item()

                optimizer_model.step()
                
                count += 1
                if i == epoch - 1:
                    real_dataset = torch.cat([real_dataset, real_label], dim=0)
                    gen_dataset = torch.cat([gen_dataset, generated_samples], dim=0)

            epoch_loss /= step_per_epoch # average loss per batch
            loss_evol.append(epoch_loss)

            print(f"epoch: {i}, Train loss: {epoch_loss:.7f}")

        return real_dataset, gen_dataset, loss_evol
    
    def train_scinet(self, model, loss_function, optim, real_train, real_test, step_per_epoch, helper):
        epoch_start = 0
        loss_evol = []
        epochs = 50
        real_dataset = torch.empty((0, self.config.future_step, self.config.input_size), dtype=torch.float32).to(self.config.device)
        gen_dataset = torch.empty((0, self.config.future_step, self.config.input_size), dtype=torch.float32).to(self.config.device)
        
        for epoch in range(epochs):
            model.train()
            total_loss = 0.0
            epoch_loss = 0.0
            count = 0
            #adjust_learning_rate(optim, epoch, self.args)

            for step in range(step_per_epoch):

                inputs = helper.gen_batch_forecasting(self.config.batch_size, count, real_train)
                targets = helper.gen_batch_forecasting(self.config.batch_size, count, real_test)

                # inference
                model.zero_grad()
                if self.config.stacks == 1:
                    forecast = model(inputs)
                    loss = loss_function(forecast, targets)
                if self.config.stacks == 2:
                    forecast, mid = model(inputs)
                    loss = loss_function(forecast, targets) + loss_function(mid, targets)
                epoch_loss += loss.item()

                # backpropagate
                loss.backward()
                total_loss += loss.item()
                optim.step()

                count += 1
                if epoch == epochs - 1:
                    real_dataset = torch.cat([real_dataset, targets], dim=0)
                    gen_dataset = torch.cat([gen_dataset, forecast], dim=0)

            epoch_loss /= step_per_epoch # average loss per batch
            loss_evol.append(epoch_loss)
            print(f"Epoch: {epoch} Loss G.: {epoch_loss}")

        return real_dataset, gen_dataset, loss_evol

In [10]:
class ModelTest():
    def __init__(self, config):
        self.config = config

    def test_model(self, model, real_data_test, data_label_test, loss_function, step_per_epoch, helper):
        model.eval()
        test_loss = []
        real_dataset = torch.empty((0, self.config.future_step, self.config.input_size), dtype=torch.float32).to(self.config.device)
        gen_dataset = torch.empty((0, self.config.future_step, self.config.input_size), dtype=torch.float32).to(self.config.device)

        with torch.no_grad():
            count = 0
            for step in range(step_per_epoch):
                real_data = helper.gen_batch_forecasting(self.config.batch_size, count, real_data_test)
                real_label = helper.gen_batch_forecasting(self.config.batch_size, count, data_label_test)
                
                generated_samples = model(real_data)
                L2 = torch.sum((real_label - generated_samples)**2)
                loss = loss_function(generated_samples, real_label) + L2
                test_loss.append(loss.item())

                real_dataset = torch.cat([real_dataset, real_label], dim=0)
                gen_dataset = torch.cat([gen_dataset, generated_samples], dim=0)

                count += 1

        return real_dataset, gen_dataset, test_loss
    
    def test_scinet_model(self, model, real_data_test, data_label_test, loss_function, step_per_epoch, helper):
        model.eval()
        test_loss = []
        real_dataset = torch.empty((0, self.config.future_step, self.config.input_size), dtype=torch.float32).to(self.config.device)
        gen_dataset = torch.empty((0, self.config.future_step, self.config.input_size), dtype=torch.float32).to(self.config.device)

        with torch.no_grad():
            count = 0
            for step in range(step_per_epoch):
                real_data = helper.gen_batch_forecasting(self.config.batch_size, count, real_data_test)
                real_label = helper.gen_batch_forecasting(self.config.batch_size, count, data_label_test)
                
                #generated_samples = model(real_data)
                if self.config.stacks == 1:
                    generated_samples = model(real_data)
                elif self.config.stacks == 2:
                    generated_samples, _ = model(real_data)
                    
                L2 = torch.sum((real_label - generated_samples)**2)
                loss = loss_function(generated_samples, real_label) + L2
                test_loss.append(loss.item())

                real_dataset = torch.cat([real_dataset, real_label], dim=0)
                gen_dataset = torch.cat([gen_dataset, generated_samples], dim=0)

                count += 1

        return real_dataset, gen_dataset, test_loss